YOLOv5 학습용 데이터를 구축하기 위해서는 이미지와 라벨 파일을 YOLO 형식으로 변환해야 합니다. 아래 코드를 사용하여 COCO 형식의 new_ann_file을 YOLO 형식으로 변환합니다.

In [ ]:
import shutil

def convert_coco_to_yolo(coco, annotations, img_dir, output_dir):
    yolo_labels_dir = os.path.join(output_dir, 'labels')
    yolo_images_dir = os.path.join(output_dir, 'images')
    os.makedirs(yolo_labels_dir, exist_ok=True)
    os.makedirs(yolo_images_dir, exist_ok=True)

    for ann in annotations:
        img = coco.loadImgs(ids=[ann['image_id']])[0]
        img_width, img_height = img['width'], img['height']
        x, y, width, height = ann['bbox']

        # Normalize bbox coordinates
        x_center = (x + width / 2) / img_width
        y_center = (y + height / 2) / img_height
        width /= img_width
        height /= img_height

        # Write YOLO label file
        label_file = os.path.join(yolo_labels_dir, f"{img['file_name'].replace('.jpg', '.txt')}")
        with open(label_file, 'a') as f:
            f.write(f"{ann['category_id']} {x_center} {y_center} {width} {height}\n")

        # Copy the image file to YOLO images folder
        src_img_file = os.path.join(img_dir, img['file_name'])
        dst_img_file = os.path.join(yolo_images_dir, img['file_name'])
        shutil.copyfile(src_img_file, dst_img_file)

# 원본 이미지 경로와 변환된 데이터를 저장할 경로를 설정합니다.
img_dir = os.path.join(data_dir, data_type)
output_dir = 'path/to/yolo/data'

# COCO 형식의 데이터를 YOLO 형식으로 변환합니다.
convert_coco_to_yolo(coco, filtered_annotations, img_dir, output_dir)


이제 YOLOv5를 사용하여 학습할 준비가 되었습니다. YOLOv5를 사용하기 위해 필요한 라이브러리와 리포지토리를 다운받아 설치합니다.

```
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install -r requirements.txt
```

학습을 위한 설정 파일을 작성합니다. 
`yolov5s.yaml`을 복사하여 새로운 파일 `custom_yolov5s.yaml`을 만들고, 클래스 개수와 데이터 경로를 수정하세요.

In [ ]:
# custom_yolov5s.yaml
nc: 2  # number of classes (원하는 클래스 개수로 변경)
# depth_multiple: 0.33  # model depth multiple
# width_multiple: 0.50  # layer channel multiple

train: path/to/yolo/data/images  # 학습용 이미지 경로
val: path/to/yolo/data/images  # 검증용 이미지 경로 (필요시 변경)

# 클래스 이름 설정
names: ['dog', 'cat']  # 원하는 클래스명으로 변경

## Train

### 학습 옵션
```
--img: 입력 이미지의 크기 (정사각형으로 가정)
--batch: 배치 크기
--epochs: 학습 에폭 수
--data: 학습 및 검증 데이터셋에 대한 설정 파일
--cfg: YOLOv5 모델 설정 파일
--weights: 사전 학습된 가중치 파일 (사전 학습된 모델을 사용하거나, 이어서 학습하려면 이전 학습의 가중치 파일 경로를 설정)
--name: 학습 결과를 저장할 디렉토리의 이름
```

In [ ]:
!python detect.py --source path/to/test/images --weights runs/train/my_yolo_model/weights/best.pt --conf 0.25 --img 640

## Inference

### 추론 옵션
```
--source: 추론할 이미지 또는 동영상의 경로, 웹캠 사용 시 '0' 입력
--weights: 학습된 가중치 파일의 경로
--conf: 객체 탐지에 대한 최소 신뢰도 임곗값
--img: 입력 이미지의 크기 (정사각형으로 가정)
````

In [ ]:
python detect.py --source path/to/test/images --weights runs/train/my_yolo_model/weights/best.pt --conf 0.25 --img 640

추론 결과는 `runs/detect` 디렉토리에 저장됩니다. 결과를 확인하려면 이 디렉토리를 열고 생성된 이미지 파일을 확인하세요.